## Lesson 5: Automate Event Planning

https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai/lesson/12/automate-event-planning-(code)

In this lesson you will learn more about Tasks.

In [1]:
import warnings

from crewai import Agent, Crew, Task
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from dotenv import find_dotenv, load_dotenv
from IPython.display import Markdown
from pydantic import BaseModel

warnings.filterwarnings("ignore")

In [2]:
load_dotenv(find_dotenv())

True

### CrewAI Tools

In [3]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

### Creating Agents

In [4]:
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue based on event requirements",
    backstory="With a keen sense of space and understanding of event logistics, "
    "you excel at finding and securing the perfect venue that"
    "fits the event's theme, size and budget constraints.",
    tools=[search_tool, scrape_tool],
    verbose=True,
)

In [5]:
logistic_manager = Agent(
    role="Logistic Manager",
    goal="Manage all logistics for the event including catering and equipment",
    backstory="Organized and detail-oriented, you ensure that every logistical aspect"
    "of the event from catering to equipment setup is flawlessly executed "
    "to create a seamless experience.",
    tools=[search_tool, scrape_tool],
    verbose=True,
)

In [6]:
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and communicate with participants",
    backstory="Creative and communicative, you craft compelling messages and "
    "engage with potential attendees to maximize event exposure and participation.",
    tools=[search_tool, scrape_tool],
    verbose=True,
)

### Creating Venue Pydantic Object
- Create a `VenueDetails` class using Pydantic's `BaseModel`
- Agents will populate this object with information about different venues by creating different instances of it.

In [7]:
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

### Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback before finalizing it

In [8]:
venue_task = Task(
    description="Find a venue in {event_city} that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen venue you found "
    "to accomadate the event",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
    agent=venue_coordinator,
)

- By setting `async_execution=True`, the task can run in parallel with the tasks which come after it.

In [9]:
logistics_task = Task(
    description="Coordinate catering and equipment for an event with {expected_participants} "
    "participants on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistic_manager,
)

In [10]:
marketing_task = Task(
    description="Promote the {event_topic} aiming to engage at least "
    "{expected_participants} potential attendees.",
    expected_output="Report on marketing activites and attendee engagement "
    "formatted as a markdown document.",
    async_execution=True,
    output_file="marketing_report.md",
    agent=marketing_communications_agent,
)

### Creating the Crew

In [11]:
event_management_crew = Crew(
    agents=[venue_coordinator, logistic_manager, marketing_communications_agent],
    tasks=[venue_task, logistics_task, marketing_task],
    verbose=True,
)

### Running the Crew

In [12]:
event_details = {
    "event_topic": "music festival",
    "event_description": "a gathering of musical artists and fans to show off their talent",
    "event_city": "London",
    "tentative_date": "2024-12-10",
    "expected_participants": 350,
    "budget": 30_000,
    "venue_type": "outdoor",
}

In [13]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in London that meets criteria for music festival.


> Entering new CrewAgentExecutor chain...
I need to find a venue in London that meets the criteria for a music festival. I should consider the theme, size, and budget constraints of the event.

Action:
Delegate work to co-worker

Action Input:
{
   "task": "Search for a venue in London suitable for a music festival with a capacity of 500 attendees and within a budget of $50,000",
   "context": "I am looking for a venue in London that meets the criteria for a music festival. The venue should be able to accommodate 500 attendees and the budget for the venue is $50,000.",
   "coworker": "Logistic Manager"
}


> Entering new CrewAgentExecutor chain...
I need to find a venue in London that can accommodate 500 attendees within a budget of $50,000 for the music festival.

Action: Search the internet
Action Input: {"venue for music festival London 500 attend

I need to gather information on the music festival, potential attendees, and marketing activities to effectively promote the event and engage with participants.

Action:
Delegate work to co-worker

Action Input:
{
  "task": "Research potential attendees for the music festival",
  "context": "I need to gather information on potential attendees to effectively promote the event and engage with participants.",
  "coworker": "Logistic Manager"
}


> Entering new CrewAgentExecutor chain...
I need to ensure that all logistics for the event, including catering and equipment setup, are flawlessly executed to create a seamless experience for the 350 participants on 2024-12-10 at The HAC in London.

Action: Delegate work to co-worker
Action Input: {"task": "Coordinate catering for the event with 350 participants on 2024-12-10 at The HAC in London. Ensure a variety of options are available to accommodate dietary restrictions and preferences.", "context": "Need to coordinate catering for the event 

In [14]:
Markdown("./marketing_report.md")

# Marketing Activities and Attendee Engagement Report

## Music Festival Promotion Strategy:
- Utilized social media platforms such as Instagram, Facebook, and Twitter to create buzz around the event.
- Collaborated with local influencers in the music industry to reach a wider audience.
- Implemented targeted email marketing campaigns to reach potential attendees.

## Attendee Engagement Activities:
- Hosted online contests and giveaways to encourage interaction and excitement among attendees.
- Provided clear and detailed information on the festival lineup, schedule, and ticket sales to keep attendees informed.
- Engaged with attendees through interactive Q&A sessions on social media platforms to build a sense of community.

## Demographics of Potential Attendees:
- Identified key demographics such as age groups, interests, and music preferences to tailor marketing messages effectively.
- Analyzed previous attendee data to understand trends and preferences to attract similar participants to the event.
- Conducted surveys and polls to gather feedback and preferences from potential attendees to improve the festival experience.

## Conclusion:
By implementing a comprehensive marketing strategy and engaging with potential attendees through various channels, the music festival successfully attracted over 350 participants. The collaboration with the Logistic Manager to research potential attendees played a crucial role in understanding the target audience and tailoring marketing efforts to maximize event exposure and participation. The festival's success can be attributed to a combination of effective promotion, engaging activities, and a deep understanding of attendee demographics.